# Data Processing

Full data dictionary available [here](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2018/cc-est2018-alldata.pdf)

In [52]:
import pandas as pd
import numpy as np

### Read in data

In [136]:
df = pd.read_csv("../data/cc-est2018-alldata-36.csv")

In [137]:
df.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,36,1,New York,Albany County,1,0,304204,147076,157128,...,5236,5416,2125,2295,361,410,139,141,41,41
1,50,36,1,New York,Albany County,1,1,15286,7821,7465,...,488,514,288,277,44,40,18,12,5,3
2,50,36,1,New York,Albany County,1,2,16131,8224,7907,...,452,443,275,258,31,36,15,11,5,3
3,50,36,1,New York,Albany County,1,3,17639,9065,8574,...,448,435,230,204,33,28,13,12,2,1
4,50,36,1,New York,Albany County,1,4,23752,11925,11827,...,632,627,271,283,50,46,16,15,4,6


## Replace Year Code Actual Dates

<span style="float:left;">
    
|YEAR Code|Dates|
|:---|:---|
|1|4/1/2010 Census population|
|2|4/1/2010 population estimates base|
|3|7/1/2010 population estimate|
|4|7/1/2011 population estimate|
|5|7/1/2012 population estimate|
|6|7/1/2013 population estimate|
|7|7/1/2014 population estimate|
|8|7/1/2015 population estimate|
|9|7/1/2016 population estimate|
|10|7/1/2017 population estimate|
|11|7/1/2018 population estimate|

In [147]:
# Dropping the population estimate in 2010 and using the Census 2010 data
df = df[~((df["YEAR"] ==2) | (df["YEAR"] ==3))]

In [148]:
df["YEAR"].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [149]:
dates = { 1:2010,
        4:2011,
        5:2012,
        6:2013,
        7:2014,
        8:2015,
        9:2016,
        10:2017,
        11:2018}

In [150]:
# replace with actual date values
df.replace({"YEAR": dates}, inplace = True)

## Filter for Asian American Demographic

<span style="float:left;">

|Code|Demographic|
|:---|:---|
|AAC_MALE|Asian alone or in combination male population|
|AAC_FEMALE|Asian alone or in combination female population|

In [152]:
# Filter for Summary Level, State Name, County Code, County Name, Year, Age Group, Total Population, and Asian American Male, Asian American Female
df = df[["SUMLEV", "STNAME", "COUNTY", "CTYNAME", "YEAR", "AGEGRP", "TOT_POP", "AAC_MALE", "AAC_FEMALE"]]

In [153]:
df.shape

(10602, 9)

## Filter for Ages 50+

<span style="float:left;">
    
|AGEGRP Code|Age|
|:---|:---|
|0|Total|
|1|Age 0 to 4 years|
|2|Age 5 to 9 years|
|3|Age 10 to 14 years|
|4|Age 15 to 19 years|
|5|Age 20 to 24 years|
|6|Age 25 to 29 years|
|7|Age 30 to 34 years|
|8|Age 35 to 39 years|
|9|Age 40 to 44 years|
|10|Age 45 to 49 years|
|11|Age 50 to 54 years|
|12|Age 55 to 59 years|
|13|Age 60 to 64 years|
|14|Age 65 to 69 years|
|15|Age 70 to 74 years|
|16|Age 75 to 79 years|
|17|Age 80 to 84 years|
|18|Age 85 years or older|

In [154]:
# Ages 50+ are marked in AGEGRP Codes 11-18
df_seniors = df.loc[df["AGEGRP"] >= 11]
df_seniors.shape

(4464, 9)

In [155]:
# Combine Male and Female into category
df_seniors["AAC"] = df_seniors["AAC_MALE"] + df_seniors["AAC_FEMALE"] 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [156]:
df_seniors.head()

,SUMLEV,STNAME,COUNTY,CTYNAME,YEAR,AGEGRP,TOT_POP,AAC_MALE,AAC_FEMALE,AAC
11,50,New York,1,Albany County,2010,11,22788,377,376,753
12,50,New York,1,Albany County,2010,12,21019,286,326,612
13,50,New York,1,Albany County,2010,13,17881,229,275,504
14,50,New York,1,Albany County,2010,14,11968,159,187,346
15,50,New York,1,Albany County,2010,15,8676,124,137,261


In [158]:
# Groupby Year Estimate
df_seniors.groupby('YEAR').sum()

,SUMLEV,COUNTY,AGEGRP,TOT_POP,AAC_MALE,AAC_FEMALE,AAC
YEAR,,,,,,,
2010,24800,30752,7192,6341539,196120,219828,415948
2011,24800,30752,7192,6498196,207638,233363,441001
2012,24800,30752,7192,6618968,216961,244320,461281
2013,24800,30752,7192,6734585,227263,256204,483467
2014,24800,30752,7192,6848525,237418,268153,505571
2015,24800,30752,7192,6946205,246928,279552,526480
2016,24800,30752,7192,7021152,255302,290159,545461
2017,24800,30752,7192,7077283,262647,299340,561987
2018,24800,30752,7192,7123270,270178,309092,579270


## Filter for New York City

New York City belongs to the countires New York, Kings, Bronx, Richmond, Queens

In [159]:
nyc = ["Bronx County", "Queens County", "New York County", "Richmond County", "Kings County"]

In [160]:
df_nyc = df_seniors[df_seniors["CTYNAME"].isin(nyc)]

In [161]:
# Groupby Year Estimate
test = df_nyc.groupby('YEAR').sum()

In [162]:
test.reset_index(inplace = True)

In [163]:
test

,YEAR,SUMLEV,COUNTY,AGEGRP,TOT_POP,AAC_MALE,AAC_FEMALE,AAC
0,2010,2000,2232,580,2424854,148080,165854,313934
1,2011,2000,2232,580,2491651,156316,175662,331978
2,2012,2000,2232,580,2544727,163231,183666,346897
3,2013,2000,2232,580,2595933,170963,192688,363651
4,2014,2000,2232,580,2647364,178577,201677,380254
5,2015,2000,2232,580,2694859,185604,210098,395702
6,2016,2000,2232,580,2731527,191619,217734,409353
7,2017,2000,2232,580,2754461,196674,224206,420880
8,2018,2000,2232,580,2773134,201561,230770,432331
